<a href="https://colab.research.google.com/github/shivams289/Projects/blob/main/Retention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

In [ ]:
transaction_df = pd.read_csv('sal.csv')

In [ ]:
transaction_df['transaction_date'] = pd.to_datetime(transaction_df.created_at, infer_datetime_format=True).dt.date

In [ ]:
transaction_df

,user_id,contact_id,created_at,revenue_amount,cost_amount,transaction_date
0,3920.0,NaN,2019-12-23 03:30:00,42000.0,36000.0,2019-12-23
1,3925.0,NaN,2019-12-23 03:42:00,155000.0,80000.0,2019-12-23
2,3925.0,NaN,2019-12-23 03:43:00,130000.0,100000.0,2019-12-23
3,3929.0,NaN,2019-12-23 04:02:00,125000.0,110000.0,2019-12-23
4,3934.0,NaN,2019-12-23 04:09:00,100000.0,75000.0,2019-12-23
...,...,...,...,...,...,...
77090,32272.0,NaN,2020-02-07 06:14:00,0.0,9000.0,2020-02-07
77091,32272.0,NaN,2020-02-07 06:14:00,0.0,10000.0,2020-02-07
77092,41875.0,29112.0,2020-02-07 06:14:00,115000.0,70000.0,2020-02-07
77093,42958.0,NaN,2020-02-07 06:15:00,20000.0,18000.0,2020-02-07


In [ ]:
# A function that will parse the date Time based cohort:  1 day of month
def get_month(x): return dt.datetime(x.year, x.month, 1) 

# Create transaction_date column based on month and store in TransactionMonth
transaction_df['TransactionMonth'] = transaction_df['transaction_date'].apply(get_month) 

# Grouping by customer_id and select the InvoiceMonth value
grouping = transaction_df.groupby('user_id')['TransactionMonth'] 

# Assigning a minimum InvoiceMonth value to the dataset
transaction_df['CohortMonth'] = grouping.transform('min')

# printing top 5 rows
print(transaction_df.head())

   user_id  contact_id           created_at  revenue_amount  cost_amount  \
0   3920.0         NaN  2019-12-23 03:30:00         42000.0      36000.0   
1   3925.0         NaN  2019-12-23 03:42:00        155000.0      80000.0   
2   3925.0         NaN  2019-12-23 03:43:00        130000.0     100000.0   
3   3929.0         NaN  2019-12-23 04:02:00        125000.0     110000.0   
4   3934.0         NaN  2019-12-23 04:09:00        100000.0      75000.0   

  transaction_date TransactionMonth CohortMonth  
0       2019-12-23       2019-12-01  2019-12-01  
1       2019-12-23       2019-12-01  2019-12-01  
2       2019-12-23       2019-12-01  2019-12-01  
3       2019-12-23       2019-12-01  2019-12-01  
4       2019-12-23       2019-12-01  2019-12-01  


In [ ]:
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

# Getting the integers for date parts from the `InvoiceDay` column
transcation_year, transaction_month, _ = get_date_int(transaction_df, 'TransactionMonth')

# Getting the integers for date parts from the `CohortDay` column
cohort_year, cohort_month, _ = get_date_int(transaction_df, 'CohortMonth')

In [ ]:
#  Get the  difference in years
years_diff = transcation_year - cohort_year

# Calculate difference in months
months_diff = transaction_month - cohort_month



transaction_df['CohortIndex'] = (years_diff * 12 + months_diff)  + 1 
print(transaction_df.head(5))

   user_id  contact_id           created_at  revenue_amount  cost_amount  \
0   3920.0         NaN  2019-12-23 03:30:00         42000.0      36000.0   
1   3925.0         NaN  2019-12-23 03:42:00        155000.0      80000.0   
2   3925.0         NaN  2019-12-23 03:43:00        130000.0     100000.0   
3   3929.0         NaN  2019-12-23 04:02:00        125000.0     110000.0   
4   3934.0         NaN  2019-12-23 04:09:00        100000.0      75000.0   

  transaction_date TransactionMonth CohortMonth  CohortIndex  
0       2019-12-23       2019-12-01  2019-12-01            1  
1       2019-12-23       2019-12-01  2019-12-01            1  
2       2019-12-23       2019-12-01  2019-12-01            1  
3       2019-12-23       2019-12-01  2019-12-01            1  
4       2019-12-23       2019-12-01  2019-12-01            1  


In [ ]:
# Counting daily active user from each chort
grouping = transaction_df.groupby(['CohortMonth', 'CohortIndex'])



# Counting number of unique customer Id's falling in each group of CohortMonth and CohortIndex
cohort_data = grouping['user_id'].apply(pd.Series.nunique)
cohort_data = cohort_data.reset_index()


 # Assigning column names to the dataframe created above
cohort_counts = cohort_data.pivot(index='CohortMonth',
                                 columns ='CohortIndex',
                                 values = 'user_id')

# Printing top 5 rows of Dataframe
cohort_data.head()

,CohortMonth,CohortIndex,user_id
0,2019-12-01,1,582
1,2019-12-01,2,96
2,2019-12-01,3,26
3,2020-01-01,1,6019
4,2020-01-01,2,785


In [ ]:
print(cohort_counts.round(1))

CohortIndex       1      2     3
CohortMonth                     
2019-12-01    582.0   96.0  26.0
2020-01-01   6019.0  785.0   NaN
2020-02-01   2311.0    NaN   NaN


In [ ]:

cohort_sizes = cohort_counts.iloc[:,0]

"""
We will divide the values in other columns with Cohort Size in order to calculate the retention rate i.e Number of Customers in Each Cohort Index
"""

retention = cohort_counts.divide(cohort_sizes, axis=0)

In [ ]:

# Coverting the retention rate into percentage and Rounding off.
retention.round(3)*100

CohortIndex,1,2,3
CohortMonth,,,
2019-12-01,100.0,16.5,4.5
2020-01-01,100.0,13.0,NaN
2020-02-01,100.0,NaN,NaN
